Data Mining Final Project 
Elona Shatri

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import nltk
import ijson
from ijson.common import ObjectBuilder
from sklearn.feature_extraction.text import CountVectorizer
import plotly.plotly as py
import plotly.graph_objs as go
from pandas.io.json import json_normalize
import math
import json
import plotly
from scipy import stats
from sklearn.cluster import KMeans
import seaborn as sns
# my functions
import Helpers.data_mining_helpers as dmh
import Helpers.text_analysis as ta
import string
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
from numpy import random
import gensim
#from sklearn.cross_validation import train_test_split
#from sklearn import linear_model
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
#from gensim.models import Word2Vec
#from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
from pandas.io.json import json_normalize
from Helpers import *


%matplotlib inline

2018-01-05 19:44:58,403 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
py.sign_in('penestia','lukQyxcPnchBptmT5evF')

Import the data from the json file which contains the data for the member info.


In [4]:
import json
with open('memberInfoForNTHU_1106_part1_RandomSelected.json',"r") as train_file:
        df = json.load(train_file)


In [5]:
df[2:3]

[{'member_info': {'chosen_video': [3913,
    3711,
    10457,
    4336,
    3762,
    9091,
    4220,
    10568],
   'course_status': {'correct': 1,
    'course_index': 69,
    'current_postid': 4220,
    'current_sentence': 1,
    'step': 3,
    'total_sentence': 134},
   'initial_action': [],
   'last_login_time': '2016-07-21 14:08:58',
   'listen_score': {'answer_ary': {'0': [{'answer': 'You know, Will Shakespeare was right: All the world is a stage.',
       'create_time': '2015-07-18 09:02:57',
       'question': None,
       'sentence_num': 0,
       'update_time': '2016-07-20 14:06:45'},
      {'answer': 'And you are always judged by the way you perform.',
       'create_time': '2015-07-18 09:04:54',
       'question': None,
       'sentence_num': 1,
       'update_time': '2015-07-18 09:04:54'},
      {'answer': "Especially in business, you've got to be able to learn how to articulate your message.",
       'create_time': '2015-07-18 14:30:30',
       'question': None,
       's

In [6]:
courses_normalized = courses_normalized.set_index('postId')

NameError: name 'courses_normalized' is not defined

In [ ]:
%%bash
grep -E '^ {2}"' memberInfoForNTHU_1106_part1_RandomSelected.json

In [ ]:
%%bash

grep -E '^ {2,6}"' memberInfoForNTHU_1106_part1_RandomSelected.json

In [ ]:
import ijson
filename = "memberInfoForNTHU_1106_part1_RandomSelected.json"
with open(filename, 'r') as f:
    objects = ijson.items(f, 'member_info.item')
    columns = list(objects)

In [ ]:
df_normalized = pd.io.json.json_normalize(df)

In [ ]:
df_normalized

In [ ]:
df_normalized.dtypes

In [ ]:
df1 = df_normalized[['member_info.vocabulary']]
 

In [ ]:
df1['words'] = df_normalized['member_info.vocabulary'].apply(lambda x: [y.get('word') for y in x])
print (df1)

In [ ]:
df4 = df1[['words']]

In [ ]:
df4

In [ ]:
df4.dtypes

In [ ]:
df4[0:2]

In [ ]:
" ".join(str(x) for x in df4)

In [ ]:
df5 = " ".join(str(x) for x in df4.words)

In [ ]:
df5[3:100]

In [ ]:
df6 = [df4.words.str.len()]
print(df6)

In [ ]:
words_saved = {}
for k, v in df6.items():
    if v > 200:
        words_saved[k] = v

In [ ]:
print(words_saved)

In [ ]:
lists = (words_saved.items())
x,y = zip(*lists)

In [ ]:
data = [go.Bar(
            x=x,
            y=y
    )]

layout = go.Layout(
    xaxis = dict(
        type = "category"
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

In [ ]:
words_count = {}
videos_words = {}
videos_unique_words = {}
member_words = {}


for x in range(0,6000):
    member_id = (df_normalized['member_info.memberid'][x])
    member_words[member_id] = 0
    for voc in (df_normalized['member_info.vocabulary'][x]):
        if voc['word'] in words_count:
            words_count[voc['word']] += 1
        else:
            words_count[voc['word']] = 1
        if voc['postID'] in videos_words:
            videos_words[voc['postID']] += 1
        else:
            videos_words[voc['postID']] = 1 
        if voc['postID'] in videos_unique_words:
            videos_unique_words[voc['postID']].add(voc['word'])
        else:
            videos_unique_words[voc['postID']] = set()
            videos_unique_words[voc['postID']].add(voc['word'])
            
    

        
        
            
        member_words[member_id] += 1

In [ ]:
member_words

In [ ]:
pkg_frequency = pd.DataFrame(list(member_words.items()), columns =['id', 'count'])
print(pkg_frequency)


In [ ]:
pkg_frequency['count'].values

In [ ]:
df8 = df_normalized[['member_info.memberid','member_info.listen_score.average']]
df8['count'] = pkg_frequency['count'].values
df8

In [ ]:
trace = go.Scattergl(
    y = df8['count'],
    x = df8['member_info.listen_score.average'],
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)
data = [trace]

layout= go.Layout(
    title= 'Chosen Video Cound and Listening Score Distribution',
    xaxis= dict(
        title= 'Listening Score Average',
        range=[0,100],
        showgrid=True,
         gridwidth=5,
    ),
    yaxis=dict(
        title= 'Chosen Video Count',
        range=[0,9000],
        showgrid=True,
        gridwidth=5,
    ),
    showlegend= False
)
fig= go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Listening_score vc ')

In [ ]:
df8.isnull().apply(lambda x: dmh.check_missing_values(x))

In [ ]:
lists = (member_words.items())
x,y = zip(*lists)

In [ ]:
data = [go.Bar(
            x=x,
            y=y
    )]

layout = go.Layout(
    xaxis = dict(
        type = "category"
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

In [ ]:
df_tr = df8

#Standardize
clmns = ['member_info.listen_score.average','count']
df_tr_std = stats.zscore(df_tr[clmns])

#Cluster the data
kmeans = KMeans(n_clusters=8, random_state=0).fit(df_tr_std)
labels = kmeans.labels_

#Glue back to originaal data
df_tr['clusters'] = labels

#Add the column into our list
clmns.extend(['clusters'])

In [ ]:
df_tr

In [ ]:

sns.lmplot('member_info.listen_score.average','count', 
           data=df_tr, 
           fit_reg=False, 
           hue="clusters",  
           scatter_kws={"marker": "D", 
                        "s": 100})
plt.title('Clusters Wattage vs Duration')
plt.xlabel('member_info.listen_score.average')
plt.ylabel('count')
